# Dekompozicije matrica

## LU dekompozicija

**LU dekompozicija** predstavlja razlaganje matrice kvadratne $A$ na proizvod donje trougaone matrice $L$ i gornje trougaone matrice $U$ u obliku $A = LU$. Na primer, jedno razlaganje matrice $3 \times 3$ može biti sledećeg oblika:

$$\begin{bmatrix}a_{11}&a_{12}&a_{13}\\a_{21}&a_{22}&a_{23}\\a_{31}&a_{32}&a_{33}\end{bmatrix} = \begin{bmatrix}l_{11}&0&0\\l_{21}&l_{22}&0\\l_{31}&l_{32}&l_{33}\end{bmatrix} \begin{bmatrix}u_{11}&u_{12}&u_{13}\\0&u_{22}&u_{23}\\0&0&u_{33}\end{bmatrix}.$$

Nema svaka kvadratna matrica LU dekompoziciju. Ukoliko je matrica invertibilna, da bi faktorizacija postojala, potrebno je da joj svi glavni minori budu različiti od nule. Ukoliko matrica nije invertibilna i ako je ranga $k$, da bi faktorizacija postojala, potrebno je da su joj prvih $k$ glavnih minora različiti od nule. Podsetimo se, glavni minor matrice reda $i$ je determinanta podmatrice koja se sastoji od prvih $i$ redova i prvih $i$ kolona.

Inače, ako nisu ispunjeni uslovi za LU dekompoziciju, svaka kvadratna matrica se može napisati u obliku $A = PLU$, gde $P$ predstavlja permutacionu matricu. Ovo razlaganje se naziva LU dekompozicija sa delimičnim pivotiranjem. Permutaciona matrica je matrica koja se sastoji samo od nula i jedinica, tako da se u svakom redu i svakoj koloni nalazi tačno jedna jedinica.

LU faktorizacija nije jedinstvena. Jedinstvenost se može postići zahtevanjem da bilo koja od matrica $L$ ili $U$ ima samo jedinice na glavnoj dijagonali.

Bibliotečka LU dekompozicija podržana je preko funkcije `lu` paketa `scipy.linalg`. Ukoliko je podešena vrednost `premulte_l` parametra na `False`, tada su povratne vrednosti permutaciona matrica $P$, donje trougaona matrica $L$ i gornje trougaona matrica $U$ tako da je $A = PLU$ i gde matrica $L$ ima jedinice na glavnoj dijagonali. Ukoliko je podešena vrednost `premulte_l` parametra na `True`, tada su povratne vrednosti permutovana matrica $L$ i gornje trougaona matrica $U$ tako da je $A = LU$. Permutovana matrica $L$ tada nastaje množenjem permutacione i donje trougaone matrice.

In [1]:
import numpy as np
import numpy.linalg as LA
import scipy.linalg as LAs

In [2]:
A = np.array([
    [1, 2, 3], 
    [2, 1, 2], 
    [4, 1, 3]
])

In [3]:
print(A)

[[1 2 3]
 [2 1 2]
 [4 1 3]]


In [4]:
P, L, U = LAs.lu(A)

In [5]:
print('P =\n', P)
print('L =\n', L)
print('U =\n', U)
print('PL =\n', np.dot(P, L))

P =
 [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
L =
 [[1.         0.         0.        ]
 [0.25       1.         0.        ]
 [0.5        0.28571429 1.        ]]
U =
 [[ 4.          1.          3.        ]
 [ 0.          1.75        2.25      ]
 [ 0.          0.         -0.14285714]]
PL =
 [[0.25       1.         0.        ]
 [0.5        0.28571429 1.        ]
 [1.         0.         0.        ]]


In [6]:
L, U = LAs.lu(A, permute_l=True)
print('L =\n', L)
print('U =\n', U)

L =
 [[0.25       1.         0.        ]
 [0.5        0.28571429 1.        ]
 [1.         0.         0.        ]]
U =
 [[ 4.          1.          3.        ]
 [ 0.          1.75        2.25      ]
 [ 0.          0.         -0.14285714]]


Primetimo da je proizvod matrica $P$ i $L$ iz prvog slučaja jednak matrici $L$ u drugom slučaju.

LU dekompozicija doprinosi efikasnijem i numerički stabilnijem rešavanju jednačina. Ako se matrica $A$ napiše u obliku $A = LU$, tada se sistem $Ax = b$ može napisati u obliku $(LU)x = L(Ux) = Ly = b$. Sada se početni sistem svodi na rešavanje sistema $Ly = b$ i $Ux = y$ koji su znatno pogodniji za rešavanje.

#### Primer.

Neka je dat sistem jednačina: 

\begin{align*}
2x_1 + x_2 + x_3 &= 4 \\
x_1 + 3x_2 + 2x_3 &= 5 \\
x_1 &= 6
\end{align*}

Rešenje pomoću LU dekompozicije:

In [7]:
A = np.array([[2, 1, 1], [1, 3, 2], [1, 0, 0]])
b = np.array([4, 5, 6])

In [8]:
L, U = LAs.lu(A, permute_l=True)
y = LAs.inv(L).dot(b)
x = LAs.inv(U).dot(y)

In [9]:
x

array([  6.,  15., -23.])

Rešenje se može dobiti i pomoću funkcije `solve` paketa `numpy.linalg`.

In [10]:
x = LA.solve(A, b)

In [11]:
x

array([  6.,  15., -23.])

Vidimo da se dobijena rešenja poklapaju.

## Čoleski dekompozicija

**Čoleski dekompozicija** predstavlja razlaganje matrice $A$ u obliku $A = LL^T$ gde je $L$ donje trougaona matrica. U nekim varijantama, vrši se razlaganje u obliku $A = U^TU$ gde je $U$ gornje trougaona matrica. Da bi razlaganje matrice moglo da se izvrši, matrica $A$ treba da bude simetrična i sa pozitivnim glavnim minorima. 

Implementirajmo funkciju koja proverava da li je matrica simertična. Funkcija `allclose` poredi vrednosti elemenata matrica do na zadatu tačnost.

In [12]:
def is_symmetric(A):
    return np.allclose(A, A.T)

 Implementirajmo sada funkciju koja proverava da li su glavni minori matrice pozitivni. 

In [13]:
def are_minors_positive(A):
    n = A.shape[0]
    
    for i in range(1, n+1):
        minor = LAs.det(A[:i, :i])
        if minor <= 0:
            return False
        
    return True

Bibliotečka Čoleski dekompozicija podržana je funkcijom `cholesky`  u oba paketa `np.linalg` i `scipy.linalg`.

Ispitajmo sada na primeru matrice 
$\begin{bmatrix}
4 & 12 & -16 \\
12 & 37 & -43 \\
-16 & -43 & 98
\end{bmatrix}$
oba uslova i izvršimo dekompoziciju ukoliko je moguće.

In [14]:
A = np.array([
    [4, 12, -16], 
    [12, 37, -43], 
    [-16, -43, 98]
])

In [15]:
print(A)

[[  4  12 -16]
 [ 12  37 -43]
 [-16 -43  98]]


In [16]:
if is_symmetric(A) and are_minors_positive(A):
    U = LAs.cholesky(A)

In [17]:
U

array([[ 2.,  6., -8.],
       [ 0.,  1.,  5.],
       [ 0.,  0.,  3.]])

Funkcija `cholesky` podržana `scipy` paketom ima podrazumevani argument `lower` postavljen na `False` što znači da vraća gornje trougaonu matricu. Postavljanjem ovog parametra na `True` povratna vrednost funkcije je donje trougaona matrica. Funkcija `cholesky` podržana `numpy` paketom ima podrazumevano (i jedino) ovakvo ponašanje.

In [18]:
if is_symmetric(A) and are_minors_positive(A):
    L = LAs.cholesky(A, lower=True)

In [19]:
L

array([[ 2.,  0.,  0.],
       [ 6.,  1.,  0.],
       [-8.,  5.,  3.]])

Provera za dobijene matrice:

In [20]:
np.dot(L, L.T)

array([[  4.,  12., -16.],
       [ 12.,  37., -43.],
       [-16., -43.,  98.]])

In [21]:
np.dot(U.T, U)

array([[  4.,  12., -16.],
       [ 12.,  37., -43.],
       [-16., -43.,  98.]])

Za matricu $3 \times 3$ izraz $A = LL^T$ se može raspisati u obliku 

$$\begin{bmatrix}a_{11}&a_{12}&a_{13}\\a_{21}&a_{22}&a_{23}\\a_{31}&a_{32}&a_{33}\end{bmatrix} = \begin{bmatrix}l_{11}&0&0\\l_{21}&l_{22}&0\\l_{31}&l_{32}&l_{33}\end{bmatrix} \begin{bmatrix}l_{11}&l_{12}&l_{13}\\0&l_{22}&l_{23}\\0&0&l_{33}\end{bmatrix}.$$

Elementi matrice $L$ za opšti slučaj se mogu izračunati pomoću sledećih formula:

$$l_{kk} = \sqrt{a_{kk} - \sum_{j=1}^{k-1}l_{kj}^2},$$

$$l_{ik} = \frac{1}{l_{kk}} \left( a_{ik} - \sum_{j=1}^{k-1} l_{ij}l_{kj} \right), i > k.$$

Sledeća funkcija implementira navedeni postupak.

In [22]:
def cholesky_formula(A):
    n = A.shape[0]
    L = np.zeros((n, n))
        
    for i in range(n):
        for k in range(i + 1):
            tmp_sum = 0
            if (i == k): 
                for j in range(k):
                    tmp_sum += L[k][j] ** 2
                L[i][k] = (A[i][i] - tmp_sum) ** 0.5
            else:
                for j in range(k):
                    tmp_sum += L[i][j] * L[k][j]
                L[i][k] = 1.0 / L[k][k] * (A[i][k] - tmp_sum)
    return L

In [23]:
L = cholesky_formula(A)

In [24]:
L

array([[ 2.,  0.,  0.],
       [ 6.,  1.,  0.],
       [-8.,  5.,  3.]])

## SVD dekompozicija

**SVD dekompozicija** predstavlja razlaganje matrice $M$ u obliku $M = USV^T$ gde je $U$ ortogonalna matrica dimenzija $m \times m$, $S$ dijagonalna matrica dimenzija $m \times n$ i $V$ ortogalna matrica dimenzija $n \times n$. Matrice $U$ i $V$ su ortogonalne, što znači zapravo da je $UU^T = I$ i $VV^T = I$. Matrica $S$ je pravougaona dijagonalna matrica, što znači da, u zavisnosti od toga koja dimenzija joj je veća, ima određen broj redova ili kolona koje se sastoje od nula.

Prethodna formula se može i vizuelizovati na sledeći način <img src="assets/svd.png">.

Matrice $U^*$ i $V^*$ označavaju matrice $U^T$ i $V^T$.

Bibliotečka dekompozicija podržana je funkcijom `svd` paketa `numpy.linalg`. Ona vraća vrednosti matrica U, vektor dijagonalnih elemenata matrice $S$ i matricu $V^T$. 

In [25]:
M = np.array([
    [1, 0, 0, 0, 2],
    [0, 0, 3, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0]
])

In [26]:
U, S, VT = LA.svd(M)

In [27]:
print('U =\n', U)
print('U * U^T =\n', np.dot(U, U.T))

U =
 [[ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0. -1.]
 [ 0.  0.  1.  0.]]
U * U^T =
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [28]:
print('S =\n', S)

S =
 [3.         2.23606798 2.         0.        ]


In [29]:
print('V^T =\n', VT)
print('V * V^T =\n', np.dot(VT.T, VT).astype('int'))

V^T =
 [[-0.          0.          1.          0.          0.        ]
 [ 0.4472136   0.          0.          0.          0.89442719]
 [-0.          1.          0.          0.          0.        ]
 [ 0.          0.          0.          1.          0.        ]
 [-0.89442719  0.          0.          0.          0.4472136 ]]
V * V^T =
 [[0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 0]]
